# MrfmSim Overview

The notebook includes everything mentioned in the overview page in the MrfmSim documentation.

The notebook saves three graph pdf drawings in the same working directory of this notebook. To turn off file saving, use `draw()`instead of ``draw(export=...)`` for the models.

In [ ]:
%pip install git+git://github.com/Marohn-Group/mmodel@develop
%pip install git+git://github.com/Marohn-Group/mrmfsim@develop
%pip install git+git://github.com/Marohn-Group/mrmfsim-marohn@develop

In [ ]:
# Load the plugins

import mrfmsim
mrfmsim.load_plugin()

In [ ]:
# define the input parameters

from mrfmsim.component import SphereMagnet, Grid, Sample, Cantilever

sample = Sample(
    spin_type="electron",
    temperature=11.0,
    T1=1.3e-3,
    T2=0.45e-6,
    spin_density=0.5,
)
magnet = SphereMagnet(radius=3300.0, mu0_Ms=440.0, origin=[0, 0, 3300])
grid = Grid(shape=[400, 1200, 8], step=[25, 25, 25], origin=[0, 0, -100])
cantilever = Cantilever(k_c=7.8e5, f_c=4.975e6)

B1 = 1.3e-3  # transverse magnetic field
mw_x_0p = 100  # zero to peak amplitude of the cantilever motion [nm]
B0 = 500  # external field [mT]
f_rf = 19e9  # microwave frequency [Hz]
h = [0, 0, 1450]  # tip sample separation [nm]


## Graph representation and metadata

In [ ]:
# import the experiments and draw the experiment graph

from mrfmsim.experiment import cermitesr

print(cermitesr)
cermitesr.draw(export="cermitesr.pdf")
# cermitesr.draw() # output in notebook only

## Model Modification

### Run time profiling

In [ ]:
from mrfmsim.shortcut import modifier_shortcut
from mrfmsim.modifier import profile_time

mod_dict = {
    "minimum absolute x offset": [profile_time()],
    "relative polarization change": [profile_time(10)],
}

cermitesr_profile = modifier_shortcut(cermitesr, mod_dict, name="CermitESR_Profile")

cermitesr_profile(B0, B1, cantilever, f_rf, grid, h, magnet, mw_x_0p, sample)

### Output values during execution

In [ ]:
from mrfmsim.shortcut import print_shortcut
from mrfmsim_marohn import UNITS

cermitesr_stdout = print_shortcut(cermitesr, ["f_rf", "dk_spin", "df_spin"], units=UNITS, name="CermitESR_Stdout")

cermitesr_stdout(B0, B1, cantilever, f_rf, grid, h, magnet, mw_x_0p, sample)

### Optimal looping

In [ ]:
from mrfmsim.shortcut import loop_shortcut

# loop f_rf

cermitesr_frf_loop = loop_shortcut(cermitesr_stdout, "f_rf", name="CermitESR_frfLoop")

cermitesr_frf_loop.draw(export="cermitesr_frf_loop.pdf")


In [ ]:
# loop B0

cermitesr_b0_frf_loop = loop_shortcut(
    cermitesr_frf_loop, "B0", name="CermitESR_b0frfLoop"
)

cermitesr_b0_frf_loop.draw(export="cermitesr_b0_frf_loop.pdf")


In [ ]:
import numpy as np

B0_list = np.arange(500, 900, 200)  # external field [mT]
f_rf_list = np.arange(18.5e9, 19.5e9, 0.5e9)  # microwave frequency [Hz]

# execute the model

cermitesr_b0_frf_loop(
    B0=B0_list,
    B1=B1,
    cantilever=cantilever,
    f_rf=f_rf_list,
    grid=grid,
    h=h,
    magnet=magnet,
    mw_x_0p=mw_x_0p,
    sample=sample,
)
